# Selected angles and edges for emotion analysis with face mesh point map from media pipe

In [1]:
import numpy as np
selected_face_mesh_points_dict = {
    0: 61,
    1: 292,
    2: 0,
    3: 17,
    4:	50,
    5:	280,
    6:	48,
    7:	4,
    8:	278,
    9:  206,
    10:	426,
    11:	133,
    12:	130,
    13:	159,
    14:	145,
    15:	362,
    16:	359,
    17:	386,
    18:	374,
    19:	122,
    20:	351,
    21:	46,
    22:	105,
    23:	107,
    24:	276,
    25:	334,
    26:	336,
}

angles_points = [
 [2, 0,3],
 [0, 2,1],
 [6, 7, 8],
 [9, 7, 10],
 [0, 7, 1],
 [1, 5, 8],
 [0, 4, 6],
 [1, 10, 8],
 [0, 9, 6],
 [13, 12, 14],
 [12, 13, 11],
 [17, 15, 18],
 [15, 17, 16],
 [21, 22, 23],
 [26, 25, 24],
 [6, 19, 23],
 [8, 20, 26],
]

selected_face_mesh_edges = []
selected_face_mesh_angles = []
for x in angles_points:
    selected_face_mesh_edges.append([selected_face_mesh_points_dict[x[0]], selected_face_mesh_points_dict[x[1]]])
    selected_face_mesh_edges.append([selected_face_mesh_points_dict[x[1]], selected_face_mesh_points_dict[x[2]]])
    selected_face_mesh_angles.append([selected_face_mesh_points_dict[y] for y in x])

def dot_product_angle(v1,v2):
    vector_dot_product = np.dot(v1,v2)
    arccos = np.arccos(vector_dot_product / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    res = np.degrees(arccos)
    return res


def get_angle(x1, x2, x3):
    v1 = [x1.x - x2.x, x1.y - x2.y, x1.z - x2.z]
    v2 = [x3.x - x2.x, x3.y - x2.y, x3.z - x2.z]
    return dot_product_angle(v1, v2)

# Detecting selected angles on video

In [ ]:
import math
from typing import List, Optional, Tuple

import cv2 as cv
import mediapipe as mp


def extract_landmarks(mediapipe_results):
    if mediapipe_results.multi_face_landmarks:
        face_landmark = mediapipe_results.multi_face_landmarks[0].landmark
        return True, [get_angle(face_landmark[face_angel[0]], face_landmark[face_angel[1]], face_landmark[face_angel[2]])
                      for face_angel in selected_face_mesh_angles]
    return False, []


def detect_landmarks(video_path, measurements_per_second = 5):
    capture = cv.VideoCapture(video_path)

    fps = capture.get(cv.CAP_PROP_FPS)
    frame_step = math.ceil(fps / measurements_per_second)
    frame_count = int(capture.get(cv.CAP_PROP_FRAME_COUNT))
    frame_ordinal = measurements_per_second

    video_landmarks: List[Optional[List[Tuple[float, float, float]]]] = []

    with mp.solutions.face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
        while True:
            if video_path:
                capture.set(cv.CAP_PROP_POS_FRAMES, frame_ordinal)
            frame_exists, frame = capture.read()

            if not frame_exists or frame_ordinal > frame_count:
                capture.release()
                cv.destroyAllWindows()
                return video_landmarks

            frame = cv.flip(frame, 1)
            rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)


            face_detected, frame_landmarks = extract_landmarks(face_mesh.process(rgb_frame))
            video_landmarks.append(frame_landmarks if face_detected else None)

            frame_ordinal += frame_step

# Analyzing RAVDESS dataset

In [ ]:
import mediapipe as mp
import os
import pandas as pd

RAVDESS_DATASET_PATH = "datasets/video/RAVDESS/"
VIDEO_FOLDERS = os.listdir(RAVDESS_DATASET_PATH)

df = pd.DataFrame()
num = 0
with mp.solutions.face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    for video_folder in VIDEO_FOLDERS:
        VIDEO_FILES = os.listdir(RAVDESS_DATASET_PATH + video_folder)
        print(video_folder)
        for file in VIDEO_FILES:
            if file[1] == '2' and file[7] != '7':
                video_frames = detect_landmarks("actors/" + video_folder + "/" + file)
                for frame_no, video_frame in enumerate(video_frames):
                        if video_frame is not None:
                            df.at[num, "frame_num"] = frame_no
                            df.at[num, "video"] = file
                            for i, angel in enumerate(selected_face_mesh_angles):
                                df.at[num, angel.__str__()] = video_frame[i]
                            num = num + 1
                        
ravdess_em_dict = {
    1: 5,
    2: 5,
    3: 0,
    4: 1,
    5: 3,
    6: 2,
    8: 4
}
df['emotion'] = df.apply(lambda x: ravdess_em_dict[int(x['video'][7])], axis=1)

df.to_csv('ravdess.csv')

# Analyzing RAF dataset

In [93]:
import cv2
import mediapipe as mp
import os

mp_face_mesh = mp.solutions.face_mesh

RAF_DATASET_PATH = "datasets/image/RAF/Image/original/"

IMAGE_FILES = os.listdir(RAF_DATASET_PATH)
import pandas as pd
df = pd.DataFrame(IMAGE_FILES)
for angle in selected_face_mesh_angles:
    df[angle.__str__()] = None

with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(RAF_DATASET_PATH + file)
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            continue
        for face_no, face_landmarks in enumerate(results.multi_face_landmarks):
            for angel in selected_face_mesh_angles:
                df.at[idx, angel.__str__()] = get_angle(face_landmarks.landmark[angel[0]], face_landmarks.landmark[angel[1]], face_landmarks.landmark[angel[2]])

df2 = pd.read_csv("datasets/image/RAF/EmoLabel/list_patition_label.txt", sep=" ", header=None)
df = df.merge(df2)
df = df[df[1] != 3]
raf_em_dict = {
    1: 4,
    2: 2,
    4: 0,
    5: 1,
    6: 3,
    7: 5
}
df['emotion'] = df.apply(lambda x: raf_em_dict[x[1]], axis=1)
df = df.dropna().reset_index(drop=True)
df = df.drop(1, axis=1)
df.to_csv('raf.csv')

# Analyzing FER dataset

In [78]:
import cv2
import mediapipe as mp
import os
import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

FER_DATASET_PATH = "datasets/image/FER/test/"
IMAGE_DIRS = os.listdir(FER_DATASET_PATH)
IMAGE_FILES = []
for element in IMAGE_DIRS:
    if not element.startswith("."):
        IMAGE_FILES = IMAGE_FILES + [element  + "/" + x for x in os.listdir(FER_DATASET_PATH + element)]

df = pd.DataFrame(IMAGE_FILES)
for angle in selected_face_mesh_angles:
    df[angle.__str__()] = None

with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        for idx, file in enumerate(IMAGE_FILES):
            image = cv2.imread(FER_DATASET_PATH + file)
            # Convert the BGR image to RGB before processing.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            # Print and draw face mesh landmarks on the image.
            if not results.multi_face_landmarks:
                continue
            for face_no, face_landmarks in enumerate(results.multi_face_landmarks):
                for angel in selected_face_mesh_angles:
                    df.at[idx, angel.__str__()] = get_angle(face_landmarks.landmark[angel[0]], face_landmarks.landmark[angel[1]], face_landmarks.landmark[angel[2]])
fer_em_dict = {
    "hap": 0,
    "sad": 1,
    "fea": 2,
    "ang": 3,
    "sur": 4,
    "neu": 5
}
df['emotion'] = df.apply(lambda x: fer_em_dict[x[0][:3]], axis=1)
df = df.dropna().reset_index(drop=True)
df.to_csv('fer_test.csv')

# Load analyzed data

In [58]:
import pandas as pd

raf_data = pd.read_csv('raf.csv', index_col=0)
raf_train = raf_data[raf_data['0'].str.startswith('tr')]
raf_test = raf_data[raf_data['0'].str.startswith('te')]
X_raf_train = raf_train.drop(['0', 'emotion'], axis=1)
y_raf_train = raf_train['emotion']
X_raf_test = raf_test.drop(['0', 'emotion'], axis=1)
y_raf_test = raf_test['emotion']

fer_train = pd.read_csv('analyzed_datasets/fer_train.csv', index_col=0)
fer_test = pd.read_csv('analyzed_datasets/fer_test.csv', index_col=0)
X_fer_train = fer_train.drop(['0', 'emotion'], axis=1)
y_fer_train = fer_train['emotion']
X_fer_test = fer_test.drop(['0', 'emotion'], axis=1)
y_fer_test = fer_test['emotion']

# Train RandomForestClassifier and LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
models_raf = [RandomForestClassifier(random_state=0, max_depth=10), LogisticRegression(max_iter=5000, random_state=0)]
models_fer = [RandomForestClassifier(random_state=0, max_depth=10), LogisticRegression(max_iter=5000, random_state=0)]
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_raf_train_oversampled, y_raf_train_oversampled = oversample.fit_resample(X_raf_train, y_raf_train)
X_fer_train_oversampled, y_fer_train_oversampled = oversample.fit_resample(X_fer_train, y_fer_train)
target_names = ['Happiness', 'Sadness', 'Fear', 'Anger', 'Surprise', 'Neutral']
for model in models_raf:
    model.fit(X_raf_train_oversampled, y_raf_train_oversampled)
    y_pred = model.predict(X_raf_test)
    print(model)
    print(classification_report(y_raf_test, y_pred, target_names=target_names))
for model in models_fer:
    model.fit(X_fer_train_oversampled, y_fer_train_oversampled)
    y_pred = model.predict(X_fer_test)
    print(model)
    print(classification_report(y_fer_test, y_pred, target_names=target_names))

# Test models on datasets other than those on which they were trained

In [ ]:
for model in models_raf:
    y_pred = model.predict(X_fer_test)
    print(model)
    print(classification_report(y_fer_test, y_pred, target_names=target_names))
for model in models_fer:
    y_pred = model.predict(X_raf_test)
    print(model)
    print(classification_report(y_raf_test, y_pred, target_names=target_names))

# Evaluate random forest on RAVDESS

In [ ]:
import numpy as np
from scipy import stats
ravdess_data = pd.read_csv('analyzed_datasets/ravdess.csv', index_col=0)
y_pred = []
for i in range(5):
    racdess_one_frame = ravdess_data[ravdess_data['frame_num'] == 7.0 + i]
    X_ravdess_one_frame = racdess_one_frame.drop(['frame_num', 'emotion', 'video'], axis=1)
    y_pred.append(models_raf[0].predict(X_ravdess_one_frame))
racdess_one_frame = ravdess_data[ravdess_data['frame_num'] == 5.0]
y_ravdess_one_frame = racdess_one_frame['emotion']
res = stats.mode(np.array(y_pred))[0]
print(classification_report(y_ravdess_one_frame, res, target_names=target_names))

# Load analyzed RAVDESS dataset

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
ravdess_data = pd.read_csv('analyzed_datasets/ravdess.csv', index_col=0)
X_ravdess_data = ravdess_data.drop(['frame_num', 'emotion', 'video'], axis=1)
y_ravdess_data = ravdess_data['emotion']
Y = pd.get_dummies(ravdess_data['emotion']).values
X_train, X_test, Y_train, Y_test = train_test_split(X_ravdess_data,Y, test_size = 0.10, random_state = 42)

# Define LSTM

In [91]:
from keras.src.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(units = 200, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.25))
model.add(LSTM(units = 200, return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(units = 50))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

# Train LSTM

In [ ]:
from keras.callbacks import EarlyStopping

epochs = 100
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
accr = model.evaluate(X_test,Y_test)

# LSTM classification report

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
y_pred = np.argmax(model.predict(X_test), axis=1)
Y_test_int = np.argmax(Y_test, axis=1)
print(classification_report(Y_test_int, y_pred))

# Convert LSTM to tensorflowjs

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "model4")